<a href="https://colab.research.google.com/github/NazarioR9/BNBR_Challenge/blob/master/mask_language_modeling/MLM_BertBase_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# References

In [1]:
# How to Fine-Tune BERT for Text Classification? (https://arxiv.org/pdf/1905.05583.pdf)
# https://github.com/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb

# Installs

In [1]:
!git clone https://github.com/NazarioR9/BNBR_Challenge.git

Cloning into 'BNBR_Challenge'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 65 (delta 31), reused 7 (delta 4), pack-reused 7
Unpacking objects: 100% (65/65), done.


In [2]:
!pip install transformers --quiet

     |████████████████████████████████| 778kB 5.0MB/s 
     |████████████████████████████████| 1.1MB 25.6MB/s 
     |████████████████████████████████| 890kB 59.6MB/s 
     |████████████████████████████████| 3.0MB 48.3MB/s 


In [3]:
# Check that we have a GPU
!nvidia-smi

Thu Jul  9 22:23:33 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Imports

In [4]:
import os, sys, gc

In [5]:
import pandas as pd

In [6]:
import torch
from transformers import BertConfig, BertTokenizerFast, BertForMaskedLM
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling

In [7]:
from transformers import pipeline

In [8]:
os.chdir('BNBR_Challenge/notebooks/')

In [13]:
!cp -r ./../bnbr/ bnbr/

In [11]:
sys.path.insert(0, './../bnbr/')

In [14]:
from bnbr.utils import seed_everything
from bnbr.data import MentalHealthDataset

Using TensorFlow backend.


# Envs

In [9]:
seed_everything(2020)

Set seed to 2020.


In [10]:
!mkdir ../mlm_finetuned_models

# Utils

In [11]:
class MLMDataset(MentalHealthDataset):
  def __init__(self, df, tokenizer):
    super(MLMDataset, self).__init__(df, task='test')
    self.tokenizer = tokenizer

  def __getitem__(self, idx):
    text, _, _ = super().__getitem__(idx)
    enc = self.tokenizer.encode(text)
    return torch.tensor(enc)

#Read 

In [12]:
path = './../data/'

In [ ]:
train = pd.read_csv(path+'final_train.csv')
test = pd.read_csv(path+'final_test.csv')

In [ ]:
train['length'] = 0
test['length'] = 0

In [ ]:
df = pd.concat([train, test])

In [ ]:
df.head()

,ID,text,label,Depression,Alcohol,Suicide,Drugs,length
0,SUAVK39Z,i feel that it was better i die am happy,0.0,1.0,0.0,0.0,0.0,0
1,9JDAGUV3,why do i get hallucinations ?,3.0,0.0,0.0,0.0,1.0,0
2,419WR1LQ,i am stressed due to lack of financial suppor...,0.0,1.0,0.0,0.0,0.0,0
3,6UY7DX6Q,why is life important ?,2.0,0.0,0.0,1.0,0.0,0
4,FYC0FTFB,how could i be helped to go through the depre...,0.0,1.0,0.0,0.0,0.0,0


#Finetuning MLM

In [15]:
model_name = 'bert-base-uncased'

In [16]:
config = BertConfig.from_pretrained(model_name)
tokenizer = BertTokenizerFast.from_pretrained(model_name, model_max_length =50)

In [17]:
model = BertForMaskedLM.from_pretrained(model_name)
# model = BertForMaskedLM(config=config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
dataset = MLMDataset(df[['text', 'length']], tokenizer)

In [19]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [22]:
training_args = TrainingArguments(
    output_dir = './',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
)

In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

### Start training

In [24]:
%%time
trainer.train()



CPU times: user 6.12 s, sys: 1.81 s, total: 7.93 s
Wall time: 8.13 s


TrainOutput(global_step=30, training_loss=2.693433165550232)

#### 🎉 Save final model

In [25]:
!mkdir fms

In [26]:
trainer.save_model("fms/")

In [ ]:
!zip -r ./../mlm_finetuned_models/mlm_bert_base_.zip fms/

  adding: fms/ (stored 0%)
  adding: fms/pytorch_model.bin (deflated 7%)
  adding: fms/training_args.bin (deflated 36%)
  adding: fms/config.json (deflated 45%)


In [ ]:
!rm -r fms/ runs/ bnbr/